# <center> 2차 크롤링 - <점핏> </center>

##### 링크들 들어가서 각각 정보 수집, 자율주행

* 기업이름
* 고용형태 : 신입, 경력
* 연봉 (대체로 면접후 결정 일겁니다, 크롤링 사이트 내용 그대로 검출하면 될듯!)
* 기업주소 : 기업 메인 위치
* 키워드 : 자율주행 / 드론 / 딥러닝 / 로봇 제어</br></br>
 -----뭉쳐서 '주요사항' column-----
* 자격 요건 : ROS, Python, C++, 학력
* 담당업무
* 우대사항

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service("../drive/chromedriver-linux64/chromedriver"), options=options)

driver.get("https://www.jumpit.co.kr/search?locationTag=1&sort=relation&keyword=%EB%A1%9C%EB%B4%87%EC%A0%9C%EC%96%B4")
time.sleep(2)

# 페이지 끝까지 스크롤
for _ in range(10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

html_source = driver.page_source
soup = BeautifulSoup(html_source, "html.parser")
time.sleep(2)

job_box_1 = soup.find('section', class_='sc-c12e57e5-3 gjgpzi')
job_box_2 = job_box_1.find('section', class_='sc-80cff0b9-0 cOzTle')
job_box_3 = job_box_2.find_all('div', class_='sc-c8169e0e-0 gOSXGP')

image_link_list = []
category_list = []
major_work_list = []

for info in job_box_3:

    job_link = info.find('a')['href']
    image_link_list.append(job_link)

    major_work = info.find('h2', class_='position_card_info_title').text
    major_work_list.append(major_work)


    category_list.append('로봇제어')

print(image_link_list, category_list)


In [2]:
print(len(image_link_list))
print(len(category_list))

52
52


In [3]:
import pandas as pd

df = pd.DataFrame({
    '키워드': category_list,
    'url': image_link_list,
    '담당업무' : major_work_list
})

In [4]:
df.tail()

,키워드,url,담당업무
47,로봇제어,/position/19192,임베디드 소프트웨어 개발
48,로봇제어,/position/10631,임베디드 MCU 펌웨어 개발자
49,로봇제어,/position/17035,F/W 펌웨어 개발
50,로봇제어,/position/12241,FA 기술엔지니어링 경력 채용
51,로봇제어,/position/9569,전장 개발 경력 채용


In [5]:
df.to_csv("../data/jumpit_robot_base.csv", sep =",", encoding = 'utf-8')

In [6]:
import pandas as pd

df = pd.read_csv("../data/jumpit_robot_base.csv", encoding='utf-8', index_col=0)

df['url'] = df['url'].apply(lambda x: f"https://www.jumpit.co.kr{x}")


In [1]:
import pandas as pd
df= pd.read_csv("../data/jumpit_robot_base.csv", encoding='utf-8', index_col=0)

In [7]:
df.tail()

,키워드,url,담당업무
47,로봇제어,https://www.jumpit.co.kr/position/19192,임베디드 소프트웨어 개발
48,로봇제어,https://www.jumpit.co.kr/position/10631,임베디드 MCU 펌웨어 개발자
49,로봇제어,https://www.jumpit.co.kr/position/17035,F/W 펌웨어 개발
50,로봇제어,https://www.jumpit.co.kr/position/12241,FA 기술엔지니어링 경력 채용
51,로봇제어,https://www.jumpit.co.kr/position/9569,전장 개발 경력 채용


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import requests
import os
from tqdm.notebook import tqdm


options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={generate_user_agent()}") 

driver = webdriver.Chrome(service=Service("../drive/chromedriver-linux64/chromedriver"),options=options)


company_address_list = []
company_name_list=[]
hire_list = []
major_work_list =[]
tech_list=[]
salary_list =[]


for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    driver.get(row['url'] )
    major_work_list_tmp = []

    time.sleep(1)  

    for _ in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    html_source = driver.page_source
    soup = BeautifulSoup(html_source, "html.parser")

    name_box = soup.find('section', class_='sc-3a5c981a-0 iORPxB')
    company_name = name_box.find('a').text
    company_name_list.append(company_name)

    company_address_box = soup.find('section', class_='sc-2da322c6-0 hWFbtF')
    company_address_box2 = company_address_box.find_all('dl', class_='sc-2da322c6-1 ksYhFT')

    company_address_list.append(company_address_box2[3].text)
    hire_list.append(company_address_box2[0].text)

    essential_tech_box = soup.find_all('dl', class_='sc-ae955bd-0 MHqtB')

    salary_list.append('회사내규에 따름')

    tech_stack = essential_tech_box[0].find('pre').text
    major_work_list_tmp.append(tech_stack)

    essential_tech1 = essential_tech_box[1].find('pre').text
    major_work_list_tmp.append(essential_tech1)

    essential_tech1 = essential_tech_box[2].find('pre').text
    major_work_list_tmp.append(essential_tech1)


    essential_tech2 = essential_tech_box[3].find('pre').text
    major_work_list_tmp.append(essential_tech2)


    tech_list.append(major_work_list_tmp)


    
    

In [14]:
print(len(company_name_list))
print(len(hire_list))
print(len(tech_list))
print(len(major_work_list))
print(len(company_address_list))

52
52
52
0
52


In [15]:
import pandas as pd

# 새로운 DataFrame 생성
new_data = pd.DataFrame({
    
    '기업이름' : company_name_list,
    '고용형태': hire_list,
    '연봉' : salary_list,
    '기업주소': company_address_list,
    '필요역량':tech_list
})

# 기존의 CSV 파일 불러오기
job_data = pd.read_csv('../data/jumpit_robot_base.csv')

# 두 DataFrame 병합
merged_data = pd.concat([job_data, new_data], axis=1)


# 결과 저장
merged_data.to_csv('../data/jumpit_robot_base2.csv', index=False)


In [16]:
merged_data.head()


,Unnamed: 0,키워드,url,담당업무,기업이름,고용형태,연봉,기업주소,필요역량
0,0,로봇제어,/position/21850,[엔젤로보틱스] 로봇 제어 엔지니어,엔젤로보틱스,경력경력 8~15년,회사내규에 따름,"근무지역서울 성동구 성수이로22길 37, 6층지도보기·주소복사","[HWC++FWSWEmbeddedC, • 모터드라이버를 이용한 로봇제어\n• 로봇제..."
1,1,로봇제어,/position/7992,자동화 시스템 및 로봇제어 소프트웨어 개발,비전세미콘,경력신입~10년,회사내규에 따름,근무지역세종특별자치시 집현중앙4로 79 (집현동)지도보기·주소복사,"[C++SW, • 자동화 시스템 및 로봇 제어 연구 및 개발, • 소프트웨어 개발 ..."
2,2,로봇제어,/position/21749,백엔드 개발,지에스티,경력경력 3~10년,회사내규에 따름,근무지역경기 부천시 석천로 397 103동 1109호지도보기·주소복사,"[GitMySQLC++AWSMSSQLPHPC#JSONGitHub, • 미디어이즈 (..."
3,3,로봇제어,/position/22482,협동로봇 제어 SW 개발자 경력사원 모집,한화로보틱스,경력경력 3~10년,회사내규에 따름,"근무지역경기 성남시 분당구 판교로 323 (삼평동,벤처포럼벤처타워) 5층지도보기·주소복사","[C++SWPythonShellEmbeddedC, • C++ 기반 협동로봇 제어 S..."
4,4,로봇제어,/position/6716,청소로봇팀 제어엔지니어,유진로봇,경력경력 5~10년,회사내규에 따름,근무지역인천 연수구 하모니로187번길 33지도보기·주소복사,"[C++EmbeddedC, • 청소로봇의 주행 제어 및 센서 퓨전 알고리즘 개발\n..."


In [17]:
# 컬럼 재정렬 및 불필요한 컬럼 제거
merged_data = merged_data[['기업이름', '고용형태', '연봉', '기업주소', '키워드', '담당업무','필요역량']]

merged_data.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,엔젤로보틱스,경력경력 8~15년,회사내규에 따름,"근무지역서울 성동구 성수이로22길 37, 6층지도보기·주소복사",로봇제어,[엔젤로보틱스] 로봇 제어 엔지니어,"[HWC++FWSWEmbeddedC, • 모터드라이버를 이용한 로봇제어\n• 로봇제..."
1,비전세미콘,경력신입~10년,회사내규에 따름,근무지역세종특별자치시 집현중앙4로 79 (집현동)지도보기·주소복사,로봇제어,자동화 시스템 및 로봇제어 소프트웨어 개발,"[C++SW, • 자동화 시스템 및 로봇 제어 연구 및 개발, • 소프트웨어 개발 ..."
2,지에스티,경력경력 3~10년,회사내규에 따름,근무지역경기 부천시 석천로 397 103동 1109호지도보기·주소복사,로봇제어,백엔드 개발,"[GitMySQLC++AWSMSSQLPHPC#JSONGitHub, • 미디어이즈 (..."
3,한화로보틱스,경력경력 3~10년,회사내규에 따름,"근무지역경기 성남시 분당구 판교로 323 (삼평동,벤처포럼벤처타워) 5층지도보기·주소복사",로봇제어,협동로봇 제어 SW 개발자 경력사원 모집,"[C++SWPythonShellEmbeddedC, • C++ 기반 협동로봇 제어 S..."
4,유진로봇,경력경력 5~10년,회사내규에 따름,근무지역인천 연수구 하모니로187번길 33지도보기·주소복사,로봇제어,청소로봇팀 제어엔지니어,"[C++EmbeddedC, • 청소로봇의 주행 제어 및 센서 퓨전 알고리즘 개발\n..."


In [18]:
# 결과 저장
merged_data.to_csv('../data/project_eda_dataset/jumpit_autonomous.csv', index=False)

In [19]:
merged_data['기업주소'] = merged_data['기업주소'].apply(lambda x: x.split('지도보기·주소복사')[0].replace('근무지역', '').strip())

# 결과 확인 및 저장
merged_data.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,엔젤로보틱스,경력경력 8~15년,회사내규에 따름,"서울 성동구 성수이로22길 37, 6층",로봇제어,[엔젤로보틱스] 로봇 제어 엔지니어,"[HWC++FWSWEmbeddedC, • 모터드라이버를 이용한 로봇제어\n• 로봇제..."
1,비전세미콘,경력신입~10년,회사내규에 따름,세종특별자치시 집현중앙4로 79 (집현동),로봇제어,자동화 시스템 및 로봇제어 소프트웨어 개발,"[C++SW, • 자동화 시스템 및 로봇 제어 연구 및 개발, • 소프트웨어 개발 ..."
2,지에스티,경력경력 3~10년,회사내규에 따름,경기 부천시 석천로 397 103동 1109호,로봇제어,백엔드 개발,"[GitMySQLC++AWSMSSQLPHPC#JSONGitHub, • 미디어이즈 (..."
3,한화로보틱스,경력경력 3~10년,회사내규에 따름,"경기 성남시 분당구 판교로 323 (삼평동,벤처포럼벤처타워) 5층",로봇제어,협동로봇 제어 SW 개발자 경력사원 모집,"[C++SWPythonShellEmbeddedC, • C++ 기반 협동로봇 제어 S..."
4,유진로봇,경력경력 5~10년,회사내규에 따름,인천 연수구 하모니로187번길 33,로봇제어,청소로봇팀 제어엔지니어,"[C++EmbeddedC, • 청소로봇의 주행 제어 및 센서 퓨전 알고리즘 개발\n..."


In [15]:
merged_data['고용형태'] = merged_data['고용형태'].str.replace('경력경력', '경력')

In [17]:
merged_data.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,엔젤로보틱스,경력 8~15년,회사내규에 따름,"서울 성동구 성수이로22길 37, 6층",로봇제어,• 모터드라이버를 이용한 로봇제어\n• 로봇제어를 위한 시스템 SW 구조설계\n• ...,"[HWC++FWSWEmbeddedC, • BLDC 제어이론 및 실무경험 보유하신 분..."
1,비전세미콘,경력신입~10년,회사내규에 따름,세종특별자치시 집현중앙4로 79 (집현동),로봇제어,• 자동화 시스템 및 로봇 제어 연구 및 개발,"[C++SW, • 자동화 시스템 또는 로봇관련 소프트웨어 개발 경험자\n• OA 능..."
2,지에스티,경력 3~10년,회사내규에 따름,경기 부천시 석천로 397 103동 1109호,로봇제어,"• 미디어이즈 ( 콘텐츠관리 , 통합관제 ) \n• 패스봇 ( 무인출입관리 로봇 키...","[GitMySQLC++AWSMSSQLPHPC#JSONGitHub, • 디지털사이니지..."
3,한화로보틱스,경력 3~10년,회사내규에 따름,"경기 성남시 분당구 판교로 323 (삼평동,벤처포럼벤처타워) 5층",로봇제어,• C++ 기반 협동로봇 제어 S/W 및 API 개발\n• 산업용 통신 프로토콜 개...,"[C++SWPythonShellEmbeddedC, • 임베디드 환경 개발 경험자\n..."
4,유진로봇,경력 5~10년,회사내규에 따름,인천 연수구 하모니로187번길 33,로봇제어,• 청소로봇의 주행 제어 및 센서 퓨전 알고리즘 개발\n• 청소로봇의 청소 로직 개...,"[C++EmbeddedC, • 시스템/컴퓨터/전자공학 전공자 우대 이동로봇 제어 및..."


In [40]:
merged_data.to_csv('../data/project_eda_dataset/jumpit_robot.csv', index=False)